---
title: Theoretical formulas for space plasma physics
subtitle: helpful in dataframes
---

default units are SI units for space plasma physics

- B : magnetic field, nT
- n : number density, cm^-3
- T : temperature, eV

Reference

- [plasmapy.formulary](https://docs.plasmapy.org/en/stable/formulary/index.html) provides theoretical formulas for calculation of physical quantities helpful for plasma physics.

In [ ]:
# | default_exp plasma.formulary
# | export

import astropy.units as u
from astropy.constants import m_p
from plasmapy.formulary import beta, Alfven_speed
from plasmapy.particles import ParticleLike
import polars as pl
import numpy as np

In [ ]:
# | export
def df_beta(
    df: pl.DataFrame,
    T: str = "T",  # temperature of the plasma
    n: str = "n",  # particle density of the plasma
    B: str = "B",  # magnetic field in the plasma,
    col_name: str = "beta",
    T_unit: u.Unit = u.eV,
    n_unit: u.Unit = u.cm**-3,
    B_unit: u.Unit = u.nT,
) -> pl.DataFrame:
    _T = df[T].to_numpy() * T_unit
    _n = df[n].to_numpy() * n_unit
    _B = df[B].to_numpy() * B_unit

    _beta = beta(T=_T, n=_n, B=_B)

    return df.with_columns(pl.Series(_beta).alias(col_name))

In [ ]:
# | export
def df_Alfven_speed(
    df: pl.DataFrame,
    B: str = "B",  # magnetic field in the plasma, could be a component, as plasmapy will take `abs` of it
    n: str = "n",  # particle density of the plasma
    ion: ParticleLike = "p+",
    col_name="Alfven_speed",
    sign=True,
    B_unit: u.Unit = u.nT,
    n_unit: u.Unit = u.cm**-3,
    speed_unit: u.Unit = u.km / u.s,
):
    _B = df[B].to_numpy() * B_unit
    _n = df[n].to_numpy() * n_unit

    _Alfven_speed = Alfven_speed(B=_B, density=_n, ion=ion).to(
        speed_unit
    )  # Always positive because of the plasmapy

    _Alfven_speed = np.sign(_B) * _Alfven_speed if sign else _Alfven_speed

    return df.with_columns(pl.Series(_Alfven_speed).alias(col_name))


def ldf_Alfven_speed(ldf: pl.LazyFrame, **kwargs):
    return ldf.collect().pipe(df_Alfven_speed, **kwargs).lazy()

In [ ]:
# | export
def thermal_spd2temp(speed, speed_unit=u.km / u.s):
    return (m_p * (speed * speed_unit) ** 2 / 2).to("eV").value


def df_thermal_spd2temp(df: pl.LazyFrame, speed_col, speed_unit=u.km / u.s):
    df = df.collect()
    return df.with_columns(
        plasma_temperature=thermal_spd2temp(df[speed_col].to_numpy(), speed_unit)
    ).lazy()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()